<a href="https://colab.research.google.com/github/Qian8988/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:

!pip install pyspark

     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=8dd1c47c50370a8aee1da2ac7234f70106453f63fd1f0174440155cde8044878
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:

!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-24 00:07:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.67MB/s    in 0.2s    

2021-10-24 00:07:13 (5.67 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [7]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
df1 = df.filter(df.total_votes >= 20)
df1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   52534548|R18VIM840CEFRP|1621360075|     143884185|The Real Kosher J...|           Books|          1|           16|        105|   N|                N|                 SLH|I'm sorry, but ca...| 2012-05-03|
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|    

In [8]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
df2 = df1.filter((df1.helpful_votes / df1.total_votes) >= 0.5)
df2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|
|         US|   52978967|R1363VA3TPNLVB|0670022969|     432940103|American Nations:...|           Books|          5|    

In [9]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
df_Y = df2.filter(df2.vine == 'Y')
df_Y.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50673938|R2Z8083V8ZFQVZ|1612183549|     561835180|Redemption (The P...|           Books|          3|           82|         87|   Y|                N|     Promising debut|The positives:  t...| 2012-05-02|
|         US|   35653914| RK25TAO4GGS5G|0547567375|     266689242|The Bell Bandit (...|           Books|          3|    

In [10]:
#retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
df_N = df2.filter(df2.vine == 'N')
df_N.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49541983|R14PMAJTY0EAAT|1937007456|     411822135|The Lost Fleet: B...|           Books|          4|          135|        142|   N|                Y|Good Continuation...|Bottom line is th...| 2012-05-03|
|         US|   52978967|R1363VA3TPNLVB|0670022969|     432940103|American Nations:...|           Books|          5|    

In [18]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import count
# Total number of reviews Paid
Y_review_count = df_Y.count()
print("Total Number of Reviews PAID : %f" % Y_review_count)


Total Number of Reviews PAID : 4781.000000


In [28]:
# Total number of reviews Unpaid
N_review_count = df_N.count()
print("Total Number of Reviews UNPAID : %f" % N_review_count)

Total Number of Reviews UNPAID : 332395.000000


In [24]:
# the number of 5-star reviews Paid
df_5star_vine = df_Y.filter(df_Y["star_rating"]=="5")
print("The Number of 5-star Reviews Paid : %f" % df_5star_vine.count())

The Number of 5-star Reviews Paid : 1604.000000


In [25]:
# the number of 5-star reviews Unpaid
df_5star_novine = df_N.filter(df_N["star_rating"]=="5")
print("The Number of 5-star Reviews Unpaid: %f" % df_5star_novine.count())

The Number of 5-star Reviews Unpaid: 168800.000000


In [34]:
#the percentage of 5-star reviews for paid 
pct_5star_vine = (df_5star_vine.count()/Y_review_count)
print("Percent of 5-Star Reviews for PAID : %f" % pct_5star_vine )

Percent of 5-Star Reviews for PAID : 0.335495


In [30]:
#the percentage of 5-star reviews for unpaid 
pct_5star_novine = (df_5star_novine.count()/N_review_count)
print("Percent of 5-Star Furniture Reviews from UNPAID : %f" % pct_5star_novine)

Percent of 5-Star Furniture Reviews from UNPAID : 0.507830
